# Hello Capstone Project Course!

#### This notebook will be mainly used for the capstone project. 

The problem we aim to solve is to analyze art museum’ locations in four of the the major World cities and find the best place for our tourist to take hotel really near mean coordinate to surround himself with the most art museum places there. Our main target are tourists with a really taste for art 

I will use the FourSquare API to collect data about locations of Art museums in 4 major world art cities which are: New York, Rome, Florence, London.

In [55]:
!pip install folium

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [69]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)

In [57]:
# type your answer here
LIMIT = 500 # Maximum is 100
#cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
cities = ["New York, NY", 'Rome, IT','Florence, IT','London, UK']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d18f941735") # ART MUSEUM ID
    results[city] = requests.get(url).json()
    #print(results[city])

In [58]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [59]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of art museum places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of art museum places in New York, NY =  100
Showing Top 100
Total number of art museum places in Rome, IT =  57
Showing Top 100
Total number of art museum places in Florence, IT =  38
Showing Top 100
Total number of art museum places in London, UK =  50
Showing Top 100


In [60]:
maps[cities[0]]

In [61]:
maps[cities[1]]

In [62]:
maps[cities[2]]

In [63]:
maps[cities[3]]

We can see that New York are the most dense city with Art Museum places. 

However, Let's have a concrete measure of this density.

For this I will use some basic statistics. I will get the mean location of the art museum places which should be near to most of them if they are really dense or far if not.

Next I will take the average of the distance of the venues to the mean coordinates.

In [64]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.03147039973805979
Rome, IT
Mean Distance from Mean coordinates
0.01788670248634824
Florence, IT
Mean Distance from Mean coordinates
0.005263975438879503
London, UK
Mean Distance from Mean coordinates
0.04527452581574577


In [65]:
maps[cities[0]]

In [66]:
maps[cities[1]]

In [67]:
maps[cities[2]]

In [68]:
maps[cities[3]]

We now see that Florence is his best option. Our tourist's best interest would be to book a hotel near that mean coordinate to surround himself with the most art museum places there!!